In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from skimage import feature
from imutils import build_montages
from imutils import paths
import numpy as np
import cv2
import os
import pickle

In [2]:
def quantify_image(img):
    features=feature.hog(img,orientations=9,pixels_per_cell=(10,10),cells_per_block=(2,2),transform_sqrt=True,block_norm="L1")
    return features




def load_split(path):
    imgp=list(paths.list_images(path))
    data=[]
    labels=[]
    for ip in imgp:
        label=ip.split(os.path.sep)[-2]
        img=cv2.imread(ip)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img=cv2.resize(img,(200,200))
        img=cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        features=quantify_image(img)
        data.append(features)
        labels.append(label)
    return (np.array(data),np.array(labels))
        

In [3]:
trp=r"spiral\spiral\training"
tep=r"spiral\spiral\testing"
(xtrain,ytrain)=load_split(trp)
(xtest,ytest)=load_split(tep)

In [4]:
le=LabelEncoder()
ytrain=le.fit_transform(ytrain)
ytest=le.transform(ytest)
print(xtrain.shape,ytrain.shape)

(72, 12996) (72,)


In [5]:
model=RandomForestClassifier(n_estimators=100)
model.fit(xtrain,ytrain)

RandomForestClassifier()

In [6]:
testp=list(paths.list_images(tep))
idxs=np.arange(0,len(testp))
idxs=np.random.choice(idxs,size=(25,),replace=False)
images=[]
for i in idxs:
    image=cv2.imread(testp[i])
    op=image.copy()
    op=cv2.resize(op,(128,128))
    img=image
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=cv2.resize(img,(200,200))
    img=cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    features=quantify_image(img)
    preds=model.predict([features])
    label=le.inverse_transform(preds)[0]
    color=(0,255,0) if label=="healthy" else (0,0,255)
    cv2.putText(op,label,(3,20),cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)
    images.append(op)
montage=build_montages(images,(128,128),(5,5))[0]
cv2.imshow("ahahaa",montage)
cv2.waitKey(0)

-1

In [7]:
predictions=model.predict(xtest)
cm=confusion_matrix(ytest,predictions).flatten()
print(cm)
(tn,fp,fn,tp)=cm
acc=(tp+tn)/float(cm.sum())
print(acc)

[14  1  3 12]
0.8666666666666667


In [52]:
pickle.dump(model,open('modelRF.pkl','wb'))

In [8]:
image=cv2.imread('test1.jpg')
op=image.copy()
op=cv2.resize(op,(128,128))
img=image
img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img=cv2.resize(img,(200,200))
img=cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
features=quantify_image(img)
preds=model.predict([features])
label=le.inverse_transform(preds)[0]
color=(0,255,0) if label=="healthy" else (0,0,255)
cv2.putText(op,label,(3,20),cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)
print(label)
cv2.imshow("hello",op)

parkinson
